In [13]:
# pip install contextily

In [ ]:
import geopandas as gpd

gdf = gpd.read_file('metropole-de-lyon_pvo_patrimoine_voirie.pvoamenagementcyclable.geojson')
gdf.head()
# gdf["typeamenagement"].unique()

In [15]:
# gdf["insee1"].unique()

In [16]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import pandas as pd

# Charger le fichier GeoJSON des aménagements cyclables
# GEOJSON
gdf = gpd.read_file('metropole-de-lyon_pvo_patrimoine_voirie.pvoamenagementcyclable.geojson')

# CSV
# gdf = gpd.read_file('pistes_lyon_comnbined.csv')

# Filtrer les données pour Paris en utilisant la colonne 'arrondissement'
gdf_lyon = gdf[gdf['commune1'].notnull()]  # Filtrer pour Lyon
# gdf_lyon = gdf_lyon[gdf_lyon['commune1'].str.contains('Lyon', case=False, na=False)]
# codes_insee_lyon = ['69003', '69034', '69069', '69081', '69085', '69088', '69089', '69091', '69096']
# gdf_lyon = gdf[gdf['insee1'].isin(codes_insee_lyon)]
gdf_lyon = gdf_lyon[gdf_lyon['commune1'].str.startswith('Lyon', na=False)]
gdf_lyon_cyclable = gdf_lyon
# Filtrer uniquement les aménagements cyclables pertinents
# aménagements_cyclables = [
#     'Bande Cyclable', 
#     'Piste Cyclable', 
#     'voie verte', 
#     'double-sens cyclable simple', 
#     'couloir bus ouvert aux vélos'
# ]

# gdf_lyon_cyclable = gdf_lyon[gdf_lyon['amenagement'].isin(aménagements_cyclables)]

### **VERIF DISTANCES**

In [ ]:
# Reprojeter en EPSG:2154 pour un calcul de longueur précis en mètres
gdf_lyon_cyclable = gdf_lyon_cyclable.to_crs(epsg=2154)
gdf_lyon_cyclable_final = gdf_lyon_cyclable
# Séparer les données selon la valeur de 'voie_a_sens_unique'
# gdf_paris_cyclable_non_unique = gdf_lyon_cyclable[gdf_lyon_cyclable['voie_a_sens_unique'] == 'non']
# gdf_paris_cyclable_unique = gdf_lyon_cyclable[gdf_lyon_cyclable['voie_a_sens_unique'] != 'non']

# Pour les aménagements à double sens, conserver les lignes les plus récentes
# gdf_paris_cyclable_non_unique = gdf_paris_cyclable_non_unique.sort_values(by='date_export', ascending=False)
# gdf_paris_cyclable_non_unique = gdf_paris_cyclable_non_unique.drop_duplicates(subset=['osm_id', 'cote_amenagement'])

# Pour les aménagements à sens unique, conserver uniquement la ligne la plus récente
# gdf_paris_cyclable_unique = gdf_paris_cyclable_unique.sort_values(by='date_export', ascending=False)
# gdf_paris_cyclable_unique = gdf_paris_cyclable_unique.drop_duplicates(subset=['osm_id'])

# Combiner les deux DataFrames traités
# gdf_paris_cyclable_final = pd.concat([gdf_paris_cyclable_non_unique, gdf_paris_cyclable_unique])

# Calculer la longueur des aménagements cyclables (en kilomètres)
gdf_lyon_cyclable_final['longueur'] = gdf_lyon_cyclable_final.geometry.length / 1000  # Conversion des mètres en kilomètres

# Grouper par type d'aménagement et sommer les longueurs
longueur_par_type = gdf_lyon_cyclable_final.groupby('typeamenagement')['longueur'].sum()

# Afficher les longueurs par type d'aménagement en kilomètres
print("Longueur par type d'aménagement cyclable à Paris (en km) :")
for typeamenagement, longueur in longueur_par_type.items():
    print(f"{typeamenagement}: {longueur:.2f} km")

# Calculer et afficher la longueur totale des aménagements cyclables (en km)
total_length = longueur_par_type.sum()
print(f"\nLongueur totale des aménagements cyclables à Lyon : {total_length:.2f} km")

### **LECTURE CARTE VIA GEOJSON**

In [ ]:
# Reprojeter en EPSG:3857 pour la visualisation sur la carte
gdf_lyon_web_mercator = gdf_lyon_cyclable_final.to_crs(epsg=3857)

# Créer une figure et un axe pour la carte
fig, ax = plt.subplots(figsize=(10, 8))

# Tracer les aménagements cyclables
gdf_lyon_web_mercator.plot(ax=ax, alpha=0.7, linewidth=1, column='typeamenagement', legend=True)

# Ajouter le fond de carte
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)

# Titre et mise en page
plt.title('Aménagements cyclables à Lyon', fontsize=16)
plt.axis('off')

# Récupérer les limites actuelles de la carte
minx, miny, maxx, maxy = gdf_lyon_web_mercator.total_bounds

# Facteur de zoom (plus grand que 1 pour zoomer)
zoom_factor = 1  # Modifie cette valeur pour plus de zoom (1.2 signifie zoom arrière, 0.8 pour zoom avant)

# Calculer les nouvelles limites avec le zoom
x_range = maxx - minx
y_range = maxy - miny

new_minx = minx - (zoom_factor - 1) * x_range / 2
new_maxx = maxx + (zoom_factor - 1) * x_range / 2
new_miny = miny - (zoom_factor - 1) * y_range / 2
new_maxy = maxy + (zoom_factor - 1) * y_range / 2

# Ajuster les limites de la carte avec le zoom
ax.set_xlim([new_minx, new_maxx])
ax.set_ylim([new_miny, new_maxy])

# Afficher la carte
plt.tight_layout()
plt.show()

In [ ]:
gdf_lyon_cyclable['typeamenagement'].unique()

In [ ]:
# gdf_paris_cyclable['osm_id'].value_counts().sort_values(ascending=False)

# gdf_lyon_cyclable[gdf_lyon_cyclable['osm_id'] == 41486595]

gdf_lyon_cyclable_final['longueur'].sum()
# gdf_lyon_cyclable.shape

### **SAVE FILE**

In [21]:
# Sauvegarder les résultats pour Lyon dans un nouveau fichier GeoJSON
# gdf_lyon_cyclable.to_file('data/paris_amenagements_cyclables.geojson', driver='GeoJSON')

# Sauvegarder les résultats pour Lyon dans un nouveau fichier CSV
gdf_lyon_cyclable_final.to_csv('data/lyon_pistes_cyclables.csv', index=False)

In [ ]:
gdf_lyon_cyclable_final['anneelivraison'].isna().sum()

In [ ]:
import geopandas as gpd
import folium

# Charger le fichier GeoJSON des aménagements cyclables
gdf = gpd.read_file('metropole-de-lyon_pvo_patrimoine_voirie.pvoamenagementcyclable.geojson')

# Filtrer les données pour Lyon
gdf_lyon = gdf[gdf['commune1'].str.startswith('Lyon', na=False)]

# Dictionnaire pour associer un type de piste à une couleur
type_to_color = {
    'Piste Cyclable': 'green',
    'Bande Cyclable': 'blue',
    'Couloir bus vélo non élargi': 'red',
    'Couloir bus vélo élargi': 'darkred',
    'Voie verte': 'orange',
    'Double sens cyclable': 'purple',
    'Chaussée à voie centrale banalisée (CVCB)': 'yellow',
    'Goulotte ou rampe': 'brown',
}

# Créer une carte centrée sur Lyon
m = folium.Map(location=[45.75, 4.85], zoom_start=13)

# Ajouter un groupe de calques pour chaque type de piste
for amenagement, color in type_to_color.items():
    # Filtrer les données pour le type de piste
    filtered_gdf = gdf_lyon[gdf_lyon['typeamenagement'] == amenagement]
    
    # Continuer seulement si le DataFrame filtré n'est pas vide
    if not filtered_gdf.empty:
        fg = folium.FeatureGroup(name=amenagement)
        
        folium.GeoJson(
            filtered_gdf[['geometry', 'typeamenagement']].to_json(),
            style_function=lambda feature, color=color: {'color': color, 'weight': 2, 'opacity': 0.6},
            tooltip=folium.GeoJsonTooltip(fields=['typeamenagement'], aliases=['Type d\'aménagement'])
        ).add_to(fg)
        
        fg.add_to(m)

# Ajouter une couche de contrôle pour activer/désactiver les pistes cyclables
folium.LayerControl().add_to(m)

# Sauvegarder la carte dans un fichier HTML
m.save('lyon_pistes_cyclables_groupes.html')

# Afficher la carte directement dans un notebook (si vous utilisez Jupyter)
m
